In [1]:
using Yao
using SymPy

In [2]:
S=YaoBlocks.ConstGate.S

S

In [3]:
@vars α
@vars β

(β,)

In [3]:
circuit = chain(
    2,
    put(2=>S'),
    put(2=>H),
    put(2=>T'),
    cnot(1,2),
    put(2=>T),
    put(2=>H),
    put(2=>S),
    cnot(2,1)
)

nqubits: 2
chain
├─ put on (2)
│  └─ Sdag
├─ put on (2)
│  └─ H
├─ put on (2)
│  └─ Tdag
├─ control(1)
│  └─ (2,) X
├─ put on (2)
│  └─ T
├─ put on (2)
│  └─ H
├─ put on (2)
│  └─ S
└─ control(2)
   └─ (1,) X


In [4]:
using StatsBase, Plots
plotly()

In [19]:
results=ArrayReg(bit"00")|>circuit|> r->measure(r, nshots=1000);
hist = fit(Histogram, Int.(results), 0:8)
bar(hist.edges[1] .- 0.5, hist.weights, legend=:none)

<!DOCTYPE html>
 
 
 Plots.jl

In [20]:
results=ArrayReg(bit"01")|>circuit|> r->measure(r, nshots=1000);
hist = fit(Histogram, Int.(results), 0:8)
bar(hist.edges[1] .- 0.5, hist.weights, legend=:none)

<!DOCTYPE html>
 
 
 Plots.jl

In [7]:
circuit|>mat|>Array

4×4 Array{Complex{Float64},2}:
 1.0+0.0im       0.0+0.0im  0.0+0.0im        0.0+0.0im
 0.0+0.0im  0.707107+0.0im  0.0+0.0im   0.707107+0.0im
 0.0+0.0im  0.707107+0.0im  0.0+0.0im  -0.707107-0.0im
 0.0+0.0im       0.0+0.0im  1.0+0.0im        0.0+0.0im

### Sweep on XY equator

In [8]:
N=2048

2048

In [9]:
function XY_state(θ)
    ket_XY=normalize!(ArrayReg(bit"00")+ArrayReg(bit"01"))|>chain(2,put(1=>Rz(θ)))
end 

XY_state (generic function with 1 method)

In [10]:
for θ in 0:π/20:π
    println(fidelity(XY_state(θ),XY_state(θ)|>circuit))
end

0.8535533905932735
0.8535533905932735
0.8535533905932737
0.8535533905932734
0.8535533905932735
0.8535533905932735
0.8535533905932735
0.8535533905932736
0.8535533905932735
0.8535533905932735
0.8535533905932735
0.8535533905932735
0.8535533905932735
0.8535533905932735
0.8535533905932735
0.8535533905932735
0.8535533905932735
0.8535533905932736
0.8535533905932737
0.8535533905932735
0.8535533905932735


### Sweep on XZ equator

In [22]:
function XZ_state(θ)
    ket_XY=normalize!(ArrayReg(bit"00")+ArrayReg(bit"01"))|>chain(2,put(1=>Ry(θ)))
end 

XZ_state (generic function with 1 method)

In [23]:
k=20
F_array = []

for θ in 0:π/k:2π
    F=fidelity(focus!(XZ_state(θ),1),focus!(XZ_state(θ)|>circuit,2))|>abs2
    push!(F_array,F)
end
scatter(0:π/k:2π,F_array)

<!DOCTYPE html>
 
 
 Plots.jl

We can modify the cloning machine to XZ equator

In [24]:
XZ_EQCM=chain(2,
    put(2,1=>Rx(-π/2)),
    circuit,
    put(2,1=>Rx(π/2)),
    put(2,2=>Rx(π/2)),
)

nqubits: 2
chain
├─ put on (1)
│  └─ rot(X, -1.5707963267948966)
├─ chain
│  ├─ put on (2)
│  │  └─ Sdag
│  ├─ put on (2)
│  │  └─ H
│  ├─ put on (2)
│  │  └─ Tdag
│  ├─ control(1)
│  │  └─ (2,) X
│  ├─ put on (2)
│  │  └─ T
│  ├─ put on (2)
│  │  └─ H
│  ├─ put on (2)
│  │  └─ S
│  └─ control(2)
│     └─ (1,) X
├─ put on (1)
│  └─ rot(X, 1.5707963267948966)
└─ put on (2)
   └─ rot(X, 1.5707963267948966)


In [25]:
k=20
F_array = []

for θ in 0:π/k:π-0.0001
    F=fidelity(focus!(XZ_state(θ),1),focus!(XZ_state(θ)|>XZ_EQCM,2))|>abs2
    println(F)
    push!(F_array,F)
end
# scatter(0:π/k:2π,F_array)

0.8535533905932731
0.8535533905932731
0.8535533905932735
0.8535533905932731
0.8535533905932731
0.8535533905932731
0.853553390593273
0.8535533905932733
0.8535533905932731
0.853553390593273
0.853553390593273
0.853553390593273
0.8535533905932731
0.8535533905932733
0.8535533905932733
0.8535533905932731
0.8535533905932731
0.8535533905932733
0.8535533905932735
0.853553390593273


In [92]:
k=20
F_array = []

for θ in 0:π/k:π-0.0001
    F=fidelity(focus!(XZ_state(θ),1),focus!(XZ_state(θ)|>XZ_EQCM,1))|>abs2
    println(F)
    push!(F_array,F)
end
# scatter(0:π/k:2π,F_array)

0.8535533905932731
0.8535533905932733
0.8535533905932733
0.8535533905932731
0.8535533905932731
0.8535533905932733
0.853553390593273
0.8535533905932735
0.8535533905932733
0.853553390593273
0.853553390593273
0.853553390593273
0.8535533905932733
0.8535533905932733
0.8535533905932735
0.8535533905932731
0.8535533905932731
0.8535533905932735
0.8535533905932733
0.8535533905932731


### Sweep on Bloch Sphere

In [33]:
num_pts = 600
indices = (0:num_pts-1).+ 0.5 

0.5:1.0:599.5

In [34]:
theta = map(acos,1 .- 2 .*indices./num_pts)
phi = π * (1 + 5^0.5) .* indices .%(2π)
coords=[]

for loc in zip(theta,phi)
    push!(coords,loc)
end

x, y, z = map(cos,phi) .* map(sin,theta), map(sin,phi) .* map(sin,theta), map(cos,theta);

In [35]:
scatter3d(x,y,z)

<!DOCTYPE html>
 
 
 Plots.jl

In [29]:
function sphere_state(loc)
    θ,ϕ=loc
    ket_XY=ArrayReg(bit"00")|>chain(2,put(1=>Ry(θ)),put(1=>Rz(ϕ)))
end 

sphere_state (generic function with 1 method)

In [36]:
F_array = []

for loc in coords
    F=fidelity(focus!(sphere_state(loc),1),focus!(sphere_state(loc)|>XZ_EQCM,2))|>abs2
    println(F)
    push!(F_array,F)
end

0.866700912536251
0.8423032213513131
0.8444169899589744
0.8843414840289809
0.8082719080865054
0.8804406789345322
0.8590911447436128
0.8089400135944534
0.9068472067486676
0.8050350173998468
0.8579026605728236
0.8933414591584469
0.7747401608388436
0.9089697434009639
0.8345958481903197
0.8131548476817598
0.9210046926970816
0.7666150725381636
0.8865315367242215
0.8805060377595444
0.7635688248292954
0.9303697409627348
0.7958814514201177
0.8387574846564049
0.9207038733962775
0.7360924895628081
0.9164357899616358
0.8508710342522793
0.7756638844685712
0.9418532297937293
0.7516387268193416
0.8749576334868124
0.9062094423939939
0.7234146214508659
0.9400473761327585
0.807164617628095
0.8077590505754686
0.942404776869709
0.7127514977859221
0.9114683494828708
0.876412238439292
0.734463888953543
0.9541705475019542
0.7552197172833005
0.8514780265289862
0.9307919694747261
0.6907443175703784
0.9408237912018725
0.8311614489662631
0.7684129707464967
0.9581423199934349
0.7051200341168105
0.896356295838721

0.7569190347925973
0.8353226362450682
0.9430493557065476
0.6775492176415477
0.9374998463239897
0.8461127928297613
0.7523750789374283
0.9590266120226355
0.7202723258523172
0.8824754589848913
0.9121302202758326
0.7014649484305773
0.9492623164188684
0.798468012233493
0.8078863039497242
0.9447249203008394
0.7075963614842745
0.9136729123864823
0.8728826262950471
0.7433098986165216
0.94918992515134
0.7621216008232061
0.8548518917471618
0.920427916625756
0.7199161268028832
0.9296074717647097
0.8325299911749406
0.7911689031509728
0.9385743988900623
0.7457964148933145
0.887685916242246
0.888671212244428
0.7512355534956503
0.9321322854503974
0.8011032774476831
0.8340196846426484
0.9182650351073153
0.7526598806346758
0.9047129851061926
0.8560790316456522
0.7911397953480093
0.9223519511614932
0.7881886662667515
0.8640722348662013
0.8910137490625512
0.7794684588207124
0.9053696594132554
0.8332809346182721
0.8284025573207805
0.9002451934756865
0.8006948015228275
0.8753433062959993
0.8638982899705565

In [37]:
scatter3d(x,y,z,zcolor=F_array./0.853)

<!DOCTYPE html>
 
 
 Plots.jl

### Symbolic on XZ

In [13]:
circuit|>mat|>Array

4×4 Array{Complex{Float64},2}:
 1.0+0.0im       0.0+0.0im  0.0+0.0im        0.0+0.0im
 0.0+0.0im  0.707107+0.0im  0.0+0.0im   0.707107+0.0im
 0.0+0.0im  0.707107+0.0im  0.0+0.0im  -0.707107-0.0im
 0.0+0.0im       0.0+0.0im  1.0+0.0im        0.0+0.0im

In [14]:
ψ=(circuit|>mat|>Array)*Array([α,β,0,0])

4-element Array{Sym,1}:
               1.0*α
 0.707106781186547*β
 0.707106781186547*β
                   0

In [15]:
ψ*ψ.transpose()

4×4 Array{Sym,2}:
               1.0*α^2  0.707106781186547*α*β  0.707106781186547*α*β  0
 0.707106781186547*α*β                0.5*β^2                0.5*β^2  0
 0.707106781186547*α*β                0.5*β^2                0.5*β^2  0
                     0                      0                      0  0

### Attack of 1024-key QKD with Economic Cloning Machine

In [143]:
using Random

In [148]:
l = 1024
keys = map(x->trunc(Int,x),rand(l).+0.5); #key=0:\ket{0} 1:\ket{1}
bases = map(x->trunc(Int,x),rand(l).+0.5); #base=0:Z , 1:X

In [149]:
function prep_state(key,base)
    if base==0
        state=ArrayReg(bit"00")
        if key==1
            return state|>chain(2,put(1=>X))
        else
            return state
        end
    
    elseif base==1
        state=normalize!(ArrayReg(bit"00")+ArrayReg(bit"01"))
        if key==1
            return state|>chain(2,put(1=>Z))
        else
            return state
        end
    end
end

prep_state (generic function with 1 method)

In [163]:
Bob_rec=[]
Eve_rec=[]
for param in zip(keys,bases)
    post_state=prep_state(param...)|>XZ_EQCM
    if param[2]==1
        post_state|> chain(2,put(1=>H),put(2=>H)) 
    end
    res=post_state|> x->measure(x, nshots=1)[1]
    push!(Bob_rec,res[1])
    push!(Eve_rec,res[2])
end

In [164]:
1-sum(map(abs,keys.-Bob_rec))/l # Fidelity of Bob

0.8720703125

In [166]:
1-sum(map(abs,keys.-Eve_rec))/l #  Fidelity of Eve

0.8603515625